# Spliting Data 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
!pip install split-folders

In [ ]:
import splitfolders
splitfolders.ratio('/content/drive/MyDrive/BE project InceptionNet v4/Aug_Data', output="/content/drive/MyDrive/BE project InceptionNet v4/data", seed=1337, ratio=(.8, 0.2)) 

In [ ]:
import warnings
warnings.filterwarnings("ignore")
!pip install pycocotools --user

In [ ]:
!python --version

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
# import the libraries as shown below
import tensorflow
from tensorflow.keras import layers
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import glob
import matplotlib.pyplot as plt
from keras.models import load_model
import ntpath
from sklearn.metrics import confusion_matrix

In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/BE project InceptionNet v4/data/train'
valid_path = '/content/drive/MyDrive/BE project InceptionNet v4/data/val'

# Using InceptionV3 as trained on ImageNet Without finetunig

In [ ]:

# Import the InceptionV3 library as shown below and add preprocessing layer to the front of InceptionV3
# Here we will be using imagenet weights

inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:

# don't train existing weights
for layer in inception.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob.glob('/content/drive/MyDrive/BE project InceptionNet v4/data/train/*')
print(len(folders))

In [ ]:
print("Building model with InceptionV3 with imagenet weights")
model = Sequential([
    inception,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.2),
    Dense(len(folders), activation='softmax')
])


model.summary()

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/BE project InceptionNet v4/data/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 50,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/BE project InceptionNet v4/data/val/',
                                            target_size = (224, 224),
                                            batch_size = 50,
                                            class_mode = 'categorical')

In [ ]:
import datetime

today = datetime.datetime.now()

print(today)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle

today = datetime.datetime.now()

filepath = "/content/drive/MyDrive/BE project InceptionNet v4/model2/model_fit_{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="acc", mode="max", patience=15)

callbacks_list = [checkpoint1, early] #early


history = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=callbacks_list
)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

# Using InceptionV3 while fine tuning the top 2 blocks

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in inception.layers[:249]:
    layer.trainable = False
for layer in inception.layers[249:]:
    layer.trainable = True

In [ ]:
print("Building model with InceptionV3 with imagenet weights")
model_finetuned = Sequential([
    inception,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(rate=0.2),
    Dense(4, activation='softmax')
])

# tell the model what cost and optimization method to use
model_finetuned.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle

filepath = "/content/drive/MyDrive/BE project InceptionNet v4/Models1/model_finetuned1_{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="acc", mode="max", patience=15)

callbacks_list = [checkpoint1, early] #early

history = model_finetuned.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set), 
    callbacks=callbacks_list
)

with open('trainHistoryDict.txt', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
best_model_finetuned = load_model('/content/drive/MyDrive/BE project InceptionNet v4/model2/model_fit_14-0.98.h5')

# Evaluation


In [ ]:
# Import OpenCV
import cv2

# Utility
import itertools
import random
from collections import Counter
from glob import iglob

test_dir='/content/drive/MyDrive/BE project InceptionNet v4/test/val'

def load_image(filename):
    img = cv2.imread(filename)
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img

classes=['bacterial', 'fungal', 'healthy', 'hypersensitivity']

def predict(image):
    probabilities = best_model_finetuned.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}

In [ ]:
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(20,20),text_size=15, norm=False, savefig=True): 
    # Create the confustion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
    n_classes = cm.shape[0] # find the number of classes we're dealing with
    
    # Plot the figure and make it pretty
    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
    fig.colorbar(cax)
    
    # Are there a list of classes?
    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])
            
    # Label the axes
    ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
    
    # Make x-axis labels appear on bottom
    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()
    
    ### Added: Rotate xticks for readability & increase font size (required due to such a large confusion matrix)
    plt.xticks(rotation=70, fontsize=text_size)
    plt.yticks(fontsize=text_size)
    
    # Set the threshold for different colors
    threshold = (cm.max() + cm.min()) / 2.
    
    # Plot the text on each cell
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if norm:
            plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
        else:
            plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
            
    # Save the figure to the current working directory
    if savefig:
        fig.savefig("confusion_matrix.png")

In [ ]:
num_of_test_samples=114

In [ ]:
Y_pred = best_model_finetuned.predict(test_set, num_of_test_samples // 50 + 1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
import os
classes=['bacterial', 'fungal', 'healthy', 'hypersensitivity']
make_confusion_matrix(test_set.classes, y_pred,classes=classes)

In [ ]:
path ='/content/drive/MyDrive/BE project InceptionNet v4/test/val/'
PList=glob.glob('/content/drive/MyDrive/BE project InceptionNet v4/test/val/*')
for filename in PList:
    img = load_image(str(filename))
    prediction = predict(img)
    print("ACTUAL CLASS: %s, PREDICTED: class: %s, confidence: %f" % (os.path.basename(filename), list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    plt.show()